# AWS Step Functions Data Science SDK で機械学習ワークフローを構築する

このノートブックは、Amazon SageMaker ノートブックインスタンスでの実行を想定しています。

機械学習プロジェクトにおいて、モデル開発の試行錯誤を効率的に行うことは非常に重要です。毎回手作業で同じような処理を行うのではなく、パラメタやソースコードを変えてパイプラインを実行するだけで試行錯誤できるようにすると、ヒューマンエラーを削減したり試行錯誤の記録を残しやすくすることができます。

AWS Step Functions Data Science SDK を使うと、AWS Step Function と Amazon SageMaker を使って、データサイエンティストが機械学習ワークフローを簡単に作成して実行することができます。詳しい情報は以下のドキュメントをご参照ください。

* [AWS Step Functions](https://aws.amazon.com/step-functions/)
* [AWS Step Functions Developer Guide](https://docs.aws.amazon.com/step-functions/latest/dg/welcome.html)
* [AWS Step Functions Data Science SDK](https://aws-step-functions-data-science-sdk.readthedocs.io)

このノートブックでは、以下の処理を行うパイプラインを作成します。
- Amazon SageMaker Processing を使ったデータの準備
- Amazon SageMaker Training を使ったモデルの学習
- Amazon SageMaker Processing を使ったモデルの評価
- AWS Lambda を使った評価結果に応じた後処理

大まかな流れは以下の通りです。

1. AWS Step Functions Data Science SDK の `ProcessingStep` を使ってデータの前処理、特徴量エンジニアリング、学習用とテスト用への分割を行う scikit-learn スクリプトを実行する SageMaker Processing Job を実行
1. AWS Step Functions Data Science SDK の `TrainingStep` を使って前処理された学習データを使ったモデルの学習を実行
1. AWS Step Functions Data Science SDK の `ProcessingStep` を使って前処理したテスト用データを使った学習済モデルの評価を実行
1. AWS Step Functions Data Science SDK の `LambdaStep`を使って最新のモデルと過去のモデルの評価指標の比較を実行


このノートブックで使用するデータは手書き数字のデータセット MNIST です。

コンテナイメージのビルドを円滑にするために、ノートブックインスタンスのストレージ容量を 30GB 程度に設定することをおすすめします。ストレージ容量は、ノートブックインスタンス作成時の追加設定の領域で設定可能です（デフォルトは 5GB）。

## Setup

このノートブックを実行するのに必要なライブラリをインストールします。

In [ ]:
# Import the latest sagemaker, stepfunctions and boto3 SDKs
import sys

!{sys.executable} -m pip install --upgrade pip
!{sys.executable} -m pip install -qU awscli boto3 "sagemaker>=2.0.0"
!{sys.executable} -m pip install -qU "stepfunctions==2.3.0"
!{sys.executable} -m pip install sagemaker-experiments
!{sys.executable} -m pip show sagemaker stepfunctions

コンテナイメージビルドの際の容量不足を回避するために以下のセルを実行して docker 関連のファイルの保存場所を変更してください。

In [ ]:
%%bash

sudo service docker stop
sudo mv /var/lib/docker /home/ec2-user/SageMaker/docker
sudo ln -s /home/ec2-user/SageMaker/docker /var/lib/docker
sudo service docker start

Table of Contents を作成する拡張機能をインストールすると便利です。以下のセルを実行したあと、このノートブックを開いているブラウザのタブをリロードすると Table of Contents の拡張機能が使えるようになります。

In [ ]:
%%sh
pip install jupyter_contrib_nbextensions
jupyter contrib nbextension install --user
jupyter nbextension enable toc2/main

### 必要なモジュールのインポート

同一アカウントで複数の方が同時にこのノートブックを実行する場合は、以下のセルの一番下の行の `user_name` に各自のお名前を設定してください。


In [ ]:
import io
import logging
import os
import random
import time

import boto3
import stepfunctions
from stepfunctions import steps
from stepfunctions.inputs import ExecutionInput
from stepfunctions.steps import (
    Chain,
    ChoiceRule,
    ModelStep,
    ProcessingStep,
    TrainingStep,
    TransformStep,
)
from stepfunctions.template import TrainingPipeline
from stepfunctions.template.utils import replace_parameters_with_jsonpath
from stepfunctions.workflow import Workflow

import sagemaker
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import image_uris
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.s3 import S3Uploader
from sagemaker.sklearn.processing import SKLearnProcessor

from sagemaker.analytics import ExperimentAnalytics
from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from smexperiments.trial_component import TrialComponent
from smexperiments.tracker import Tracker

from datetime import datetime
from dateutil import tz
from time import sleep

# SageMaker Session
sagemaker_session = sagemaker.Session()
region = sagemaker_session.boto_region_name

# SageMaker Execution Role
# You can use sagemaker.get_execution_role() if running inside sagemaker's notebook instance
role = get_execution_role()

account_id = boto3.client('sts').get_caller_identity().get('Account')
bucket = sagemaker_session.default_bucket()
iam_client = boto3.client('iam', region_name=region)
lambda_client = boto3.client('lambda', region_name=region)
ecr_client = boto3.client('ecr', region_name=region)
JST = tz.gettz('Asia/Tokyo')
project_name = 'pytorch-mnist'
user_name = 'sample'
container_image_list = []

次に、ノートブックから Step Functions を実行するための IAM ロール設定を行います。

### ノートブックインスタンスの IAM ロールに権限を追加

以下の手順を実行して、ノートブックインスタンスに紐づけられた IAM ロールに、AWS Step Functions のワークフローを作成して実行するための権限と Amazon ECR にイメージを push するための権限を追加してください。


1. [Amazon SageMaker console](https://console.aws.amazon.com/sagemaker/) を開く
1. **ノートブックインスタンス** を開いて現在使用しているノートブックインスタンスを選択する
1. **アクセス許可と暗号化** の部分に表示されている IAM ロールへのリンクをクリックする
1. IAM ロールの ARN は後で使用するのでメモ帳などにコピーしておく
1. **ポリシーをアタッチします** をクリックして `IAMFullAccess` を検索する
1. `IAMFullAccess` の横のチェックボックスをオンにする
1. **ポリシーのアタッチ** をクリックする


もしこのノートブックを SageMaker のノートブックインスタンス以外で実行している場合、その環境で AWS CLI 設定を行ってください。詳細は [Configuring the AWS CLI](https://docs.aws.amazon.com/cli/latest/userguide/cli-chap-configure.html) をご参照ください。

In [ ]:
policy_list = ['AWSStepFunctionsFullAccess', 'AmazonEC2ContainerRegistryFullAccess', 'AWSLambda_FullAccess']

for l in policy_list:
    response = iam_client.attach_role_policy(
        RoleName=role.split('/')[2],
        PolicyArn='arn:aws:iam::aws:policy/' + l
    )

次に Step Functions で使用する実行ロールを作成します。

### Step Functions の実行ロールの作成

作成した Step Functions ワークフローは、AWS の他のサービスと連携するための IAM ロールを必要とします。以下のセルを実行して、必要な権限を持つ IAM Policy を作成し、それを新たに作成した IAM Role にアタッチします。

なお、今回は広めの権限を持つ IAM Policy を作成しますが、ベストプラクティスとしては必要なリソースのアクセス権限と必要なアクションのみを有効にします。

In [ ]:
import json

step_functions_policy_name = 'AmazonSageMaker-StepFunctionsWorkflowExecutionPolicy-' + user_name
inline_policy ={
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "VisualEditor0",
            "Effect": "Allow",
            "Action": [
                "events:PutTargets",
                "events:DescribeRule",
                "events:PutRule"
            ],
            "Resource": [
                "arn:aws:events:*:*:rule/StepFunctionsGetEventsForSageMakerTrainingJobsRule",
                "arn:aws:events:*:*:rule/StepFunctionsGetEventsForSageMakerTransformJobsRule",
                "arn:aws:events:*:*:rule/StepFunctionsGetEventsForSageMakerTuningJobsRule",
                "arn:aws:events:*:*:rule/StepFunctionsGetEventsForECSTaskRule",
                "arn:aws:events:*:*:rule/StepFunctionsGetEventsForBatchJobsRule"
            ]
        },
        {
            "Sid": "VisualEditor1",
            "Effect": "Allow",
            "Action": "iam:PassRole",
            "Resource": role,
            "Condition": {
                "StringEquals": {
                    "iam:PassedToService": "sagemaker.amazonaws.com"
                }
            }
        },
        {
            "Sid": "VisualEditor2",
            "Effect": "Allow",
            "Action": [
                "ecs:DescribeTasks",
                "ecs:RunTask",
                "ecs:StopTask",
                "lambda:InvokeFunction",
                "sagemaker:CreateEndpoint",
                "sagemaker:CreateEndpointConfig",
                "sagemaker:CreateHyperParameterTuningJob",
                "sagemaker:CreateModel",
                "sagemaker:CreateProcessingJob",
                "sagemaker:CreateTrainingJob",
                "sagemaker:CreateTransformJob",
                "sagemaker:DeleteEndpoint",
                "sagemaker:DeleteEndpointConfig",
                "sagemaker:DescribeHyperParameterTuningJob",
                "sagemaker:DescribeProcessingJob",
                "sagemaker:DescribeTrainingJob",
                "sagemaker:DescribeTransformJob",
                "sagemaker:ListProcessingJobs",
                "sagemaker:ListTags",
                "sagemaker:StopHyperParameterTuningJob",
                "sagemaker:StopProcessingJob",
                "sagemaker:StopTrainingJob",
                "sagemaker:StopTransformJob",
                "sagemaker:UpdateEndpoint",
                "sns:Publish",
                "sqs:SendMessage"
            ],
            "Resource": "*"
        }
    ]
}

response = iam_client.create_policy(
    PolicyName=step_functions_policy_name,
    PolicyDocument=json.dumps(inline_policy),
)

step_functions_policy_arn = response['Policy']['Arn']

In [ ]:
step_functions_role_name = 'AmazonSageMaker-StepFunctionsWorkflowExecutionRole-' + user_name
assume_role_policy = {
      "Version": "2012-10-17",
      "Statement": [{"Sid": "","Effect": "Allow","Principal": {"Service":"states.amazonaws.com"},"Action": "sts:AssumeRole"}]
    }
response = iam_client.create_role(
    Path = '/service-role/',
    RoleName = step_functions_role_name,
    AssumeRolePolicyDocument = json.dumps(assume_role_policy),
    MaxSessionDuration=3600*12 # 12 hours
)

workflow_execution_role = response['Role']['Arn']

response = iam_client.attach_role_policy(
    RoleName=step_functions_role_name,
    PolicyArn=step_functions_policy_arn
)

response = iam_client.attach_role_policy(
    RoleName=step_functions_role_name,
    PolicyArn='arn:aws:iam::aws:policy/CloudWatchEventsFullAccess'
)
workflow_execution_role

### Step Functions ワークフロー実行時の入力スキーマ作成

Step Functions ワークフローを実行する際に、パラメタなどを引数として渡すことができます。これにより、パラメタのみを変えてワークフローを実行する際は、ワークフローを作り直さなくてもすみます。ここではそれらの引数のスキーマを作成します。

In [ ]:
# SageMaker expects unique names for each job, model and endpoint.
# If these names are not unique the execution will fail. Pass these
# dynamically for each execution using placeholders.
execution_input = ExecutionInput(
    schema={
        "PreprocessingJobName": str,
#         "PreprocessingInputData": str,
        "PreprocessingCode": str,
        "PreprocessingOutputData": str,
        "TrainingJobName": str,
        "TrainingParameters": dict,
        "TrainingOutputModel": str,
        "ExperimentName": str,
        "EvaluationScriptPath": str,
        "EvaluationProcessingJobName": str,
        "EvaluationProcessingOutput": str,
        "EvaluationExperimentArgs": list,
    }
)

### Amazon SageMaker Experiments のセットアップ

このノートブックでは、モデルの評価メトリクスを記録するために Amazon SageMaker Experiments を使用します。以下のセルを実行してください。

In [ ]:
experiment_name = project_name + '-' + user_name

# create the experiment if it doesn't exist
try:
    experiment_evaluate = Experiment.load(experiment_name=experiment_name)
except Exception as ex:
    if "ResourceNotFound" in str(ex):
        experiment_evaluate = Experiment.create(
            experiment_name=experiment_name, 
            description="model evaluation", 
            sagemaker_boto_client=boto3.client('sagemaker'))

print(experiment_evaluate.experiment_name)

### ベースイメージを Amazon ECR に push

試行錯誤の中で何度もコンテナイメージのビルドを繰り返していると、Docker Hub からベースイメージを pull できなくなることがあります。そこで、はじめにベースイメージをビルドして Amazon ECR に push しておき、そちらを今後ベースイメージとして使用します。

In [ ]:
base_repository = project_name + '-base-' + user_name
tag = ':latest'
uri_suffix = 'amazonaws.com'
base_repository_uri = '{}.dkr.ecr.{}.{}/{}'.format(account_id, region, uri_suffix, base_repository + tag)

!docker build -t $base_repository .
!$(aws ecr get-login --region $region --registry-ids $account_id --no-include-email)
!aws ecr create-repository --repository-name $base_repository
!docker tag {base_repository + tag} $base_repository_uri
!docker push $base_repository_uri

## データの準備

AWS が Amazon S3 上に用意した MNIST データセットを学習ジョブで使用できる形に加工します。データ準備スクリプトを実行するために `ScriptProcessor`を作成します。これは、任意のコンテナイメージを使って Processing ジョブを実行するためのクラスです。

データ準備スクリプトでは以下の処理を実行します。

* Amazon S3 からデータをダウンロード
* ダウンロードしたデータを解凍
* データを pt 形式にして S3 にアップロード

学習スクリプトでは、準備した学習用データとラベル情報を使用してモデルを学習します。また、モデル評価スクリプトでは学習済みモデルと準備したデータとラベル情報を使用してモデルを評価します。

### データ準備用コンテナイメージの作成

まずは MNIST データセットを準備してモデルの学習に使える状態にします。このサンプルノートブックでは、データセットを作成するために torchvision などを使用するため、それらのライブラリを使えるコンテナイメージを作成する必要があります。

In [ ]:
dockerfile_name = 'docker/data-preparation/Dockerfile'

dockerfile_code = f"""FROM {base_repository_uri}
    
ENV AWS_DEFAULT_REGION {region}

COPY requirements.txt .
RUN pip3 install --upgrade pip
RUN pip3 install -qU -r requirements.txt

ENTRYPOINT ["python3", "/opt/ml/processing/input/code/preprocessing.py"]
"""
with open(dockerfile_name, 'w') as f:
    f.write(dockerfile_code)

以下のセルを実行してコンテナイメージのビルドと Amazon ECR への push を行います。ログの最後に Pushed と書かれていることを確認してください。Pushed というログがない場合、イメージのビルドや push に失敗している可能性があります。

In [ ]:
ecr_repository = project_name + '-preparation-' + user_name
container_image_list.append(ecr_repository)
tag = ':latest'
uri_suffix = 'amazonaws.com'
processing_repository_uri = '{}.dkr.ecr.{}.{}/{}'.format(account_id, region, uri_suffix, ecr_repository + tag)

# Create ECR repository and push docker image
!docker build -t $ecr_repository docker/data-preparation
!$(aws ecr get-login --region $region --registry-ids $account_id --no-include-email)
!aws ecr create-repository --repository-name $ecr_repository
!docker tag {ecr_repository + tag} $processing_repository_uri
!docker push $processing_repository_uri

以下のセルを実行して、前処理のための Processing Job を実行するための `ScriptProcessor` を作成します。

In [ ]:
from sagemaker.processing import ScriptProcessor

data_preparation_processor = ScriptProcessor(
    command=['python3'],
    image_uri=processing_repository_uri,
    role=role,
    sagemaker_session=sagemaker_session,
    instance_count=1,
    instance_type='ml.m5.xlarge',
    max_runtime_in_seconds=1200
)

データ準備スクリプトを S3 にアップロードします。パイプライン（Step Functions ワークフロー）実行時にデータ準備用スクリプトのパスを指定することで、ジョブ起動後にそのパスからデータ準備スクリプトが Processing 用コンテナにダウンロードされます。

In [ ]:
timestamp = datetime.now(tz=JST).strftime('%Y%m%-d%H%M')

PREPROCESSING_SCRIPT_LOCATION = "preprocessing.py"

preparation_input_code = sagemaker_session.upload_data(
    PREPROCESSING_SCRIPT_LOCATION,
    bucket=bucket,
    key_prefix=os.path.join(project_name, user_name, "data_preparation/code", timestamp),
)
print('preparation_input_code: ', preparation_input_code)

### `ProcessingStep` の作成

それでは、Step Functions ワークフローから SageMaker Processing ジョブを起動するための [ProcessingStep](https://aws-step-functions-data-science-sdk.readthedocs.io/en/stable/sagemaker.html#stepfunctions.steps.sagemaker.ProcessingStep) を作成しましょう。

このステップは、前の手順で定義した ScriptProcessor に入力と出力の情報を追加して使用します。

#### [ProcessingInputs](https://sagemaker.readthedocs.io/en/stable/api/training/processing.html#sagemaker.processing.ProcessingInput) と [ProcessingOutputs](https://sagemaker.readthedocs.io/en/stable/api/training/processing.html#sagemaker.processing.ProcessingOutput)  オブジェクトを作成して SageMaker Processing ジョブに入力と出力の情報を追加

In [ ]:
inputs = [
#     ProcessingInput(
#         source=execution_input["PreprocessingInputData"], destination="/opt/ml/processing/input", input_name="input-1"
#     ),
    ProcessingInput(
#         source=input_code,
        source=execution_input["PreprocessingCode"],
        destination="/opt/ml/processing/input/code",
        input_name="code",
    ),
]

processing_output_path = '/opt/ml/processing/output'
outputs = [
    ProcessingOutput(
        source=processing_output_path,
        destination=execution_input["PreprocessingOutputData"],
        output_name="prepared_data",
    )
]

####  `ProcessingStep` の作成

In [ ]:
processing_step = ProcessingStep(
    "Data Preparation Step",
    processor=data_preparation_processor,
    job_name=execution_input["PreprocessingJobName"],
    inputs=inputs,
    outputs=outputs,
    container_arguments=["--output-dir", processing_output_path],
    container_entrypoint=["python3", "/opt/ml/processing/input/code/preprocessing.py"],
)

## 準備したデータを使ったモデルの学習


### 学習用コンテナイメージの作成
まずは、モデルの学習で使用するコンテナイメージを作成します。以下のセルと実行して Dockerfile を作成します。

In [ ]:
filepath = 'docker/train/Dockerfile'

train_dockerfile={f"""FROM 763104351884.dkr.ecr.{region}.amazonaws.com/pytorch-training:1.8.1-gpu-py36-cu111-ubuntu18.04
 
COPY requirements.txt ./
RUN pip uninstall torch torchvision -y && pip install --no-cache-dir -U -r requirements.txt

ENV SAGEMAKER_PROGRAM train.py
"""
}

with open(filepath, 'w') as f:
    f.write('\n'.join(list(train_dockerfile)))

コンテナイメージをビルドして Amazon ECR に push します。このセルの実行が完了するまでに 5分ほどかかります。

In [ ]:
ecr_repository = project_name + '-train-' + user_name
container_image_list.append(ecr_repository)
tag = ':latest'
uri_suffix = 'amazonaws.com'
train_repository_uri = '{}.dkr.ecr.{}.{}/{}'.format(account_id, region, uri_suffix, ecr_repository + tag)

# Create ECR repository and push docker image
!docker build -t $ecr_repository docker/train
!aws ecr get-login-password --region {region} | docker login --username AWS --password-stdin 763104351884.dkr.ecr.{region}.amazonaws.com
!aws ecr get-login-password --region {region} | docker login --username AWS --password-stdin {account_id}.dkr.ecr.{region}.amazonaws.com
!aws ecr create-repository --repository-name $ecr_repository
!docker tag {ecr_repository + tag} $train_repository_uri
!docker push $train_repository_uri

学習スクリプト `train.py` を使って学習ジョブを実行するための `PyTorch` インスタンスを作成します。これはあとで `TrainingStep` を作成する際に使用します。

In [ ]:
from sagemaker.pytorch import PyTorch

metric_definitions = [{'Name': 'average test loss',
                       'Regex': 'Test set: Average loss: ([0-9\\.]+)'}]

estimator = PyTorch(entry_point='train.py',
                    image_uri=train_repository_uri,
                    role=role,
#                     framework_version='1.8.0',  # image_uri ではなく framework_version を指定すると組み込みコンテナを利用可能
#                     py_version='py3',
                    instance_count=1,
                    instance_type='ml.m5.xlarge',
                    metric_definitions=metric_definitions,
                    hyperparameters={
                        'batch-size':128,
                        'lr': 0.01,
                        'epochs': 1,
                        'backend': 'gloo'
                    })

学習スクリプト `train.py` は、ロジスティック回帰モデルを学習し、学習済みモデルを `/opt/ml/model` に保存します。Amazon SageMaker は、学習ジョブの最後に `/opt/ml/model` に保存されているモデルを `model.tar.gz` に圧縮して S3 にアップロードします。学習スクリプトは基本的に、ローカル環境で使用する際のものを流用できますが、SageMaker で使えるように argparse を使って各種変数を受け取る記述を追加してください。

学習用スクリプトを source.tar.gz に固めて S3 にアップロードします。既存のファイルを上書きしてしまわないよう、タイムスタンプを使ってアップロード先のパスを指定します。

In [ ]:
timestamp = datetime.now(tz=JST).strftime('%Y%m%d-%H%M')

TRAINNING_SCRIPT_LOCATION = "source.tar.gz"
!tar zcvf $TRAINNING_SCRIPT_LOCATION train.py

train_code = sagemaker_session.upload_data(
    TRAINNING_SCRIPT_LOCATION,
    bucket=bucket,
    key_prefix=os.path.join(project_name, user_name, "train/code", timestamp),
)
train_code

### `TrainingStep` の作成

Step Function ワークフローから学習ジョブを起動するために `TrainingStep` を作成します。

In [ ]:
training_step = steps.TrainingStep(
    "SageMaker Training Step",
    estimator=estimator,
    data={"training": sagemaker.TrainingInput(execution_input["PreprocessingOutputData"])},
    job_name=execution_input["TrainingJobName"],
    hyperparameters=execution_input["TrainingParameters"],
#     experiment_config={
#              "ExperimentName":execution_input["ExperimentName"]},
    wait_for_completion=True,
)

## モデルの評価

`evaluation.py` はモデル評価用のスクリプトです。このスクリプトは scikit-learn と Amazon SageMake Experiments を用いるため、カスタムコンテナを利用できる`ScriptProcessor` を使用します。このスクリプトは学習済みモデルとテスト用データセットを入力として受け取り、loss と accuracy が記載された JSON ファイルを出力します。

パラメタや学習データを変えて複数のモデルを学習させ、それらの精度を比較する際に評価メトリクスの情報を Amazon SageMaker Experiments で管理しておくと、作成した複数のモデルを比較しやすくなります。以下のモデル評価用のスクリプトでは、モデルの評価メトリクスを算出してそれらを Experiment に登録し、また、最新の学習済みモデルが既存のモデルと比べて良いかどうかを評価メトリクスを使って判定し、その結果も Experiment に記録しています。学習ジョブや Processing ジョブの中で Experiments の Tracker を使う際は、`Tracker.load() ` を使用することでジョブが使用している Trial を自動的に読み出して tracking を開始します。

評価用スクリプトを S3 にアップロードします。既存のファイルを上書きしてしまわないよう、タイムスタンプを使ってアップロード先のパスを指定します。

In [ ]:
timestamp = datetime.now(tz=JST).strftime('%Y%m%d-%H%M')

MODELEVALUATION_SCRIPT_LOCATION = "evaluation.py"

input_evaluation_code = sagemaker_session.upload_data(
    MODELEVALUATION_SCRIPT_LOCATION,
    bucket=bucket,
    key_prefix=os.path.join(project_name, user_name, "evaluation/code", timestamp),
)
print('input_evaluation_code:', input_evaluation_code)

モデル評価用の ProcessingStep の入力と出力オブジェクトを作成します。

In [ ]:
eval_model_dir = "/opt/ml/processing/model"
eval_data_dir = "/opt/ml/processing/data"
inputs_evaluation = [
    # データ準備 steo で準備したデータの設定（評価データ）
    ProcessingInput(
        source=execution_input["PreprocessingOutputData"],
        destination=eval_data_dir,
        input_name="input-1",
    ),
    # 評価対象の学習済みモデルの設定
    ProcessingInput(
        source=execution_input["TrainingOutputModel"],
        destination=eval_model_dir,
        input_name="input-2",
    ),
    # 評価スクリプトの設定
    ProcessingInput(
        source=execution_input['EvaluationScriptPath'],
        destination="/opt/ml/processing/input/code",
        input_name="code",
    ),
]

outputs_evaluation = [
    # 評価結果の設定
    ProcessingOutput(
        source="/opt/ml/processing/evaluation",
        destination=execution_input["EvaluationProcessingOutput"],
        output_name="evaluation",
    ),
]

モデルの評価結果を SageMaker Experiments で管理するため、Experiments のライブラリを含んだコンテナイメージを作成します。

In [ ]:
dockerfile_name = 'docker/model-evaluation/Dockerfile'

dockerfile_code = f"""FROM {base_repository_uri}
    
ENV AWS_DEFAULT_REGION {region}

COPY requirements.txt .
RUN pip3 install --upgrade pip
RUN pip3 install -qU -r requirements.txt

ENTRYPOINT ["python3", "/opt/ml/processing/input/code/evaluation.py"]
"""
with open(dockerfile_name, 'w') as f:
    f.write(dockerfile_code)

In [ ]:
ecr_repository = project_name + '-evaluate-' + user_name
container_image_list.append(ecr_repository)
tag = ':latest'
uri_suffix = 'amazonaws.com'
evaluate_repository_uri = '{}.dkr.ecr.{}.{}/{}'.format(account_id, region, uri_suffix, ecr_repository + tag)

# Create ECR repository and push docker image
!docker build -t $ecr_repository docker/model-evaluation
!$(aws ecr get-login --region $region --registry-ids $account_id --no-include-email)
!aws ecr create-repository --repository-name $ecr_repository
!docker tag {ecr_repository + tag} $evaluate_repository_uri
!docker push $evaluate_repository_uri

データの前処理の際は SKProcessor を使用しましたが、モデルの評価にはカスタムコンテナを使用するため ScriptProcessor を使用します。

In [ ]:
from sagemaker.processing import ScriptProcessor

model_evaluation_processor = ScriptProcessor(
    command=['python3'],
    image_uri=evaluate_repository_uri,
    role=role,
    sagemaker_session=sagemaker_session,
    instance_count=1,
    instance_type='ml.m5.xlarge',
    max_runtime_in_seconds=1200
)

### `ProcessingStep` の作成

ProcessingStep の引数 experiment_config に Experiment 名をセットすることで、起動した Processing ジョブの中で自動的に Trial が作成されます。

In [ ]:
processing_evaluation_step = ProcessingStep(
    "SageMaker Processing Model Evaluation step",
    processor=model_evaluation_processor,
    job_name=execution_input["EvaluationProcessingJobName"],
    inputs=inputs_evaluation,
    outputs=outputs_evaluation,
    experiment_config={
             "ExperimentName":execution_input["ExperimentName"]},
    container_arguments=execution_input["EvaluationExperimentArgs"],
    container_entrypoint=["python3", "/opt/ml/processing/input/code/evaluation.py"],
)

## モデルの評価結果に応じた処理

モデルの評価ジョブでの評価結果をもとに、Lambda 関数で後処理を行います。後処理としては例えば、Slack にモデルの評価結果をポストしたり、別の Workflow を起動したりなどが考えられます。このノートブックでは、評価結果に応じて表示するテキストの内容を変えています。

実行環境やソースコードが入ったコンテナイメージを Lambda にデプロイします。このノートブックでは、Experiment で管理されている情報を取得するために `sagemaker.analytics.ExperimentAnalytics` を使用するため、必要なライブラリが入ったコンテナを使って Lambda 関数を実行します。

In [ ]:
dockerfile_name = 'docker/postprocess-lambda/Dockerfile'

dockerfile_code = f"""FROM {base_repository_uri}

# Include global arg in this stage of the build
ARG FUNCTION_DIR="/function"

COPY requirements.txt .
RUN pip3 install --upgrade pip
RUN pip3 install -qU -r requirements.txt

# Copy function code
RUN mkdir -p $FUNCTION_DIR
COPY app/ $FUNCTION_DIR/

# Set working directory to function root directory
WORKDIR $FUNCTION_DIR


ENTRYPOINT [ "/usr/local/bin/python", "-m", "awslambdaric" ]
CMD [ "app.handler" ]
"""

with open(dockerfile_name, 'w') as f:
    f.write(dockerfile_code)

In [ ]:
ecr_repository_lambda = project_name + '-lambda-' + user_name
container_image_list.append(ecr_repository_lambda)
tag = ':latest'
uri_suffix = 'amazonaws.com'
lambda_repository_uri = '{}.dkr.ecr.{}.{}/{}'.format(account_id, region, uri_suffix, ecr_repository_lambda + tag)

# Create ECR repository and push docker image
!docker build -t $ecr_repository_lambda docker/postprocess-lambda
!$(aws ecr get-login --region $region --registry-ids $account_id --no-include-email)
!aws ecr create-repository --repository-name $ecr_repository_lambda
!docker tag {ecr_repository_lambda + tag} $lambda_repository_uri
!docker push $lambda_repository_uri

### コンテナイメージを使って AWS Lambda 関数を作成

Lambda で使用するコンテナイメージができたら、コンテナイメージを使って Lambda 関数を作成します。ここからは、必要な権限を持つ IAM Role/Policy の作成と Lambda 関数の作成を行います。

In [ ]:
import boto3
import json

def create_container_lambda_function(function_name, image_uri, policy_list=[], trust_service_list=[]):

    timestamp = datetime.now(tz=JST).strftime('%Y%m%d%H%M')
    lambda_function_name = function_name
    lambda_inference_policy_name = lambda_function_name + '-policy-'+timestamp
    lambda_inference_role_name = lambda_function_name + '-role-'+timestamp

    inline_policy = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Sid": "VisualEditor0",
                "Effect": "Allow",
                "Action": "sagemaker:Search",
                "Resource": "*"
            },
            {
                'Effect': 'Allow',
                'Action': 'logs:CreateLogGroup',
                'Resource': f'arn:aws:logs:{region}:{account_id}:*'
            },
            {
                'Effect': 'Allow',
                'Action': [
                    'logs:CreateLogStream',
                    'logs:PutLogEvents'
                ],
                'Resource': [
                    f'arn:aws:logs:{region}:{account_id}:log-group:/aws/lambda/{lambda_function_name}:*'
                ]
            }
        ]
    }

    response = iam_client.create_policy(
        PolicyName=lambda_inference_policy_name,
        PolicyDocument=json.dumps(inline_policy),
    )

    policy_arn = response['Policy']['Arn']

    service_list = ["lambda.amazonaws.com"]
    for t in trust_service_list:
        service_list.append(t)
    
    assume_role_policy = {
      "Version": "2012-10-17",
      "Statement": [{"Sid": "","Effect": "Allow","Principal": {"Service":service_list},"Action": "sts:AssumeRole"}]
    }
    response = iam_client.create_role(
        Path = '/service-role/',
        RoleName = lambda_inference_role_name,
        AssumeRolePolicyDocument = json.dumps(assume_role_policy),
        MaxSessionDuration=3600*12 # 12 hours
    )
    
    lambda_role_arn = response['Role']['Arn']
    lambda_role_name = response['Role']['RoleName']
    
    response = iam_client.attach_role_policy(
        RoleName=lambda_inference_role_name,
        PolicyArn=policy_arn
    )
    
    for p in policy_list:
        arn = 'arn:aws:iam::aws:policy/' + p
        response = iam_client.attach_role_policy(
            RoleName=lambda_inference_role_name,
            PolicyArn=arn
        )
    
    sleep(20) # wait until IAM is created
    
    response = lambda_client.create_function(
        FunctionName=function_name,
        Role=lambda_role_arn,
        Code={
            'ImageUri':image_uri
        },
        Timeout=60*5, # 5 minutes
        MemorySize=128, # 128 MB
        Publish=True,
        PackageType='Image',
    )
    
    return lambda_role_name, policy_arn

In [ ]:
lambda_function_name_eval = project_name + '-evaluate-model-' + user_name
lambda_role, lambda_policy = create_container_lambda_function(lambda_function_name_eval, lambda_repository_uri)

既存の Lambda 関数で使用するコンテナイメージを更新する場合は、以下のセルのコメントアウトを外して実行してください。新しいコンテナイメージが反映
されるまで2分ほどお待ちください。

In [ ]:
# response = lambda_client.update_function_code(
#         FunctionName=lambda_function_name_eval,
#         ImageUri=lambda_repository_uri
#     )

### LambdaStep の作成

作成した Lambda 関数を使って `LambdaStep` を作成します。

In [ ]:
from stepfunctions.steps.states import Retry
lambda_step = stepfunctions.steps.compute.LambdaStep(
    "Query Evaluation Results",
    parameters={
        "FunctionName": lambda_function_name_eval,
        "Payload": {
            "experiment-name": execution_input["ExperimentName"],
             "evaluation-job-name": execution_input["EvaluationProcessingJobName"],
        },
    },
)
lambda_step.add_retry(
    Retry(error_equals=["States.TaskFailed"], interval_seconds=15, max_attempts=2, backoff_rate=4.0)
)

## Fail 状態の作成
いずれかのステップが失敗したときにワークフローが失敗だとわかるように `Fail` 状態を作成します。

In [ ]:
failed_state_sagemaker_processing_failure = stepfunctions.steps.states.Fail(
    "ML Workflow failed", cause="SageMakerProcessingJobFailed"
)

#### ワークフローの中のエラーハンドリングを追加

エラーハンドリングのために [Catch Block](https://aws-step-functions-data-science-sdk.readthedocs.io/en/stable/states.html#stepfunctions.steps.states.Catch) を使用します。もし いずれかの Step が失敗したら、`Fail` 状態に遷移します。

In [ ]:
catch_state_processing = stepfunctions.steps.states.Catch(
    error_equals=["States.TaskFailed"],
    next_step=failed_state_sagemaker_processing_failure,
)

processing_step.add_catch(catch_state_processing)
processing_evaluation_step.add_catch(catch_state_processing)
training_step.add_catch(catch_state_processing)
lambda_step.add_catch(catch_state_processing)

## `Workflow` の作成と実行

ここまでで Step Functions のワークフローを作成する準備が完了しました。それでは、ワークフローを作成して実行してみましょう。

Chain を使って各 Step を連結してワークフローを作成します。既存のワークフローを変更する場合は、update() を実行します。ログに ERROR が表示された場合は、以下のセルを再度実行してください。

In [ ]:
workflow_graph = Chain([processing_step, training_step, processing_evaluation_step, lambda_step])
# workflow_graph = Chain([processing_step, training_step, processing_evaluation_step])

branching_workflow = Workflow(
    name=project_name + '-' + user_name,
    definition=workflow_graph,
    role=workflow_execution_role,
)

branching_workflow.create()
branching_workflow.update(workflow_graph)

### SageMaker ジョブ名の作成

ワークフローを繰り返し実行したい場合は、以下のセルから `branching_workflow.execute()` のセルまでの 3つのセルを 1セットとして実行してください。SageMaker のジョブ名は重複させることができないため、timestamp を使ってジョブ名を生成してからワークフローを実行する必要があります。

In [ ]:
timestamp = datetime.now(tz=JST).strftime('%Y%m%d-%H%M%S')

# Generate unique names for Pre-Processing Job, Training Job, and Model Evaluation Job for the Step Functions Workflow
training_job_name = f"{project_name}-training-{user_name}-{timestamp}"  # Each Training Job requires a unique name
preprocessing_job_name = f"{project_name}-preparation-{user_name}-{timestamp}"  # Each Preprocessing job requires a unique name,
evaluation_job_name = f"{project_name}-evaluation-{user_name}-{timestamp}"  # Each Evaluation Job requires a unique name
timestamp

### Processing ジョブの出力パスの作成

Processing ジョブの出力を保存する S3 パスを作成します。

In [ ]:
# 前処理ジョブ用
s3_bucket_base_uri = "{}{}".format("s3://", bucket)
# output_data = "{}/{}/{}-{}".format(s3_bucket_base_uri, user_name, "data/sklearn_processing/output", timestamp)
output_data = "{}/{}".format(s3_bucket_base_uri, preprocessing_job_name)
prepared_training_data = 0

# モデル評価ジョブ用
preprocessed_testing_data = "{}/{}".format(output_data, "test_data")
model_data_s3_uri = "{}/{}/{}".format(s3_bucket_base_uri, training_job_name, "output/model.tar.gz")
output_model_evaluation_s3_uri = "{}/{}/{}".format(
    s3_bucket_base_uri, training_job_name, "evaluation"
)

### ワークフローの実行

パラメタを指定して、ワークフローを実行します。

In [ ]:
# Execute workflow
execution = branching_workflow.execute(
    inputs={
        "PreprocessingJobName": preprocessing_job_name,  # Each pre processing job (SageMaker processing job) requires a unique name,
#         "PreprocessingInputData": input_data,
        "PreprocessingCode": preparation_input_code,
        "PreprocessingOutputData": output_data,
        "TrainingJobName": training_job_name,  # Each Sagemaker Training job requires a unique name,
        "TrainingParameters": {
                                     "sagemaker_program": "train.py",
                                     "sagemaker_submit_directory":  train_code,
                                      "epochs": '1'
        },
        "TrainingOutputModel": model_data_s3_uri,
        "ExperimentName": experiment_evaluate.experiment_name,
        "EvaluationScriptPath": input_evaluation_code,
        "EvaluationProcessingJobName": evaluation_job_name,  # Each SageMaker processing job requires a unique name,
        "EvaluationProcessingOutput": output_model_evaluation_s3_uri,
        "EvaluationExperimentArgs": ['--experiment-name', experiment_evaluate.experiment_name,
                                     '--model-dir', eval_model_dir,
                                     '--data-dir', eval_data_dir]
    }
)

以下のセルを実行することで、Workflow の進行状況がわかります。実行開始から12分程度で完了します。

In [ ]:
execution.render_progress()

### ワークフローの出力を確認

Amazon S3 から `evaluation.json` を取得して確認します。ここにはモデルの評価レポートが書かれています。なお、以下のセルは Step Functions でワークフローの実行が完了してから（`evaluation.json` が出力されてから）実行してください。

In [ ]:
workflow_execution_output_json = execution.get_output(wait=True)

In [ ]:
from sagemaker.s3 import S3Downloader
import json

evaluation_s3_uri = output_model_evaluation_s3_uri + '/evaluation.json'
evaluation_output = S3Downloader.read_file(evaluation_s3_uri)
evaluation_output_dict = json.loads(evaluation_output)
print(json.dumps(evaluation_output_dict, sort_keys=True, indent=4))

## Amazon SageMaker Experiments の確認

Workflow の実行が完了したら、Experiment の中をのぞいてみましょう。まずは Experiment のデータを ExperimentAnalytics を使って読み出します。

In [ ]:
search_expression = {
    "Filters":[
        {
            "Name": "TrialComponentName",
            "Operator": "Contains",
            "Value": evaluation_job_name,
        }
    ],
}

trial_component_analytics = ExperimentAnalytics(
    experiment_name=experiment_evaluate.experiment_name,
    sort_by="parameters.accuracy",
#     search_expression=search_expression,
#     sort_by="metrics.acc.max",
#     sort_order="Ascending",# Ascending or Descending
#     metric_names=['metric1', 'metric2'],
#     parameter_names=['accuracy', 'roc_auc'],
    input_artifact_names=[]
)

読み出したデータを DataFrame 形式に変換して表示します。accuracy や average_loss が記録されていることがわかります。

In [ ]:
import pandas as pd
df = trial_component_analytics.dataframe()
pd.set_option('display.max_columns', None)
df

表示が省略されて見にくいので、DataFrame を HTML ファイルに変換して保存します。以下のセルを実行すると、このノートブックと同じパスに experiments.html というファイルが作成されます。

In [ ]:
pd.set_option('colheader_justify', 'center') 

html_string = '''
<html>
  <head><title>Experiments</title></head>
  <body>
    {table}
  </body>
</html>.
'''

# OUTPUT AN HTML FILE
with open('experiments.html', 'w') as f:
    f.write(html_string.format(table=df.to_html()))

ノートブックインスタンスでこのノートブックを実行している場合、以下のセルを実行して experiments.html の内容を確認できます。SageMaker Studio を使用している場合は、直接 html ファイルをクリックして確認してください。

In [ ]:
from IPython.display import HTML
HTML('experiments.html')

## Step Functions Workflow の実行のみ行う場合

ここまでで、Step Functions Data Science SDK を使って Workflow を作成し、実行するところまでご紹介しました。実際に使用する際は、作成済みの Workflow をパラメタを指定して実行する部分を繰り返すことになります。ここでは、作成済みの Workflow を実行する部分のみを抜き出してご紹介します。

Workflow の ARN を使って既存の Workflow を読み出します。ARN はわからないけれど Workflow 名はわかる、という場合は以下のセルを実行することで ARN を取得することができます。（`workflow_name` に Workflow 名を入れてください）

In [ ]:
workflow_name = project_name + '-' + user_name
workflow_list = Workflow.list_workflows()
workflow_arn = [d['stateMachineArn'] for d in workflow_list  if d['name']==workflow_name][0]
workflow_arn

Workflow を読み出して `execute` で実行します。実行する際にパラメタを指定することができます。SageMaker ジョブはユニークな名前である必要があるため、このノートブックでは datetime を使ってジョブ名を生成しています。学習に使用するコードを変更する場合は、使用したいスクリプト（source.tar.gz）をアップロードしてある S3 パスを以下のセルの `train_code` に設定し、コメントアウトを解除してから実行してください。

In [ ]:
from datetime import datetime
from dateutil import tz
from stepfunctions.workflow import Workflow

JST = tz.gettz('Asia/Tokyo')
region = sagemaker_session.boto_region_name
# input_data = "s3://sagemaker-sample-data-{}/processing/census/census-income.csv".format(region)
# train_code = 's3://xxx/code/source.tar.gz'

bucket = sagemaker_session.default_bucket()

timestamp = datetime.now(tz=JST).strftime('%Y%m%d-%H%M%S')

# Generate unique names for Pre-Processing Job, Training Job, and Model Evaluation Job for the Step Functions Workflow
training_job_name = f"{project_name}-training-{user_name}-{timestamp}"  # Each Training Job requires a unique name
preprocessing_job_name = f"{project_name}-preparation-{user_name}-{timestamp}"  # Each Preprocessing job requires a unique name,
evaluation_job_name = f"{project_name}-evaluation-{user_name}-{timestamp}"  # Each Evaluation Job requires a unique name

# 前処理ジョブ用
s3_bucket_base_uri = "{}{}".format("s3://", bucket)
output_data = "{}/{}".format(s3_bucket_base_uri, preprocessing_job_name)
prepared_training_data = 0

# モデル評価ジョブ用
preprocessed_testing_data = "{}/{}".format(output_data, "test_data")
model_data_s3_uri = "{}/{}/{}".format(s3_bucket_base_uri, training_job_name, "output/model.tar.gz")
output_model_evaluation_s3_uri = "{}/{}/{}".format(
    s3_bucket_base_uri, training_job_name, "evaluation"
)

existing_workflow = Workflow.attach(workflow_arn)
execution = existing_workflow.execute(
    inputs={
        "PreprocessingJobName": preprocessing_job_name,  # Each pre processing job (SageMaker processing job) requires a unique name,
#         "PreprocessingInputData": input_data,
        "PreprocessingCode": preparation_input_code,
        "PreprocessingOutputData": output_data,
        "TrainingJobName": training_job_name,  # Each Sagemaker Training job requires a unique name,
        "TrainingParameters": {
                                     "sagemaker_program": "train.py",
                                     "sagemaker_submit_directory":  train_code,
                                      "epochs": '1'
        },
        "TrainingOutputModel": model_data_s3_uri,
        "ExperimentName": experiment_evaluate.experiment_name,
        "EvaluationScriptPath": input_evaluation_code,
        "EvaluationProcessingJobName": evaluation_job_name,  # Each SageMaker processing job requires a unique name,
        "EvaluationProcessingOutput": output_model_evaluation_s3_uri,
        "EvaluationExperimentArgs": ['--experiment-name', experiment_evaluate.experiment_name,
                                     '--model-dir', eval_model_dir,
                                     '--data-dir', eval_data_dir]
    }
)

以下のセルを実行することで、Workflow の進行状況がわかります。

In [ ]:
execution.render_progress()

## リソースの削除

このノートブックの実行が終わったら、不要なリソースを削除することを忘れないでください。このノートブックを最後まで実行してリソースの削除をしたら、ノートブックインスタンス、各種データを保存した S3 バケットも不要であれば削除してください。

### Step Functions Workflow の削除

In [ ]:
branching_workflow.delete()

def detach_role_policies(role_name):
    response = iam_client.list_attached_role_policies(
        RoleName=role_name,
    )
    policies = response['AttachedPolicies']

    for p in policies:
        response = iam_client.detach_role_policy(
            RoleName=role_name,
            PolicyArn=p['PolicyArn']
        )
    
detach_role_policies(step_functions_role_name)
iam_client.delete_role(RoleName=step_functions_role_name)
iam_client.delete_policy(PolicyArn=step_functions_policy_arn)

### Experiment の削除

In [ ]:
sm = boto3.Session().client('sagemaker')
def cleanup(experiment):
    for trial_summary in experiment.list_trials():
        trial = Trial.load(sagemaker_boto_client=sm, trial_name=trial_summary.trial_name)
        for trial_component_summary in trial.list_trial_components():
            tc = TrialComponent.load(
                sagemaker_boto_client=sm,
                trial_component_name=trial_component_summary.trial_component_name)
            trial.remove_trial_component(tc)
            try:
                # comment out to keep trial components
                tc.delete()
            except:
                # tc is associated with another trial
                continue
            # to prevent throttling
            time.sleep(.5)
        trial.delete()
    experiment.delete()

In [ ]:
cleanup(experiment_evaluate)

### Lambda 関数の削除

In [ ]:
lambda_role, lambda_policy 

detach_role_policies(lambda_role)
iam_client.delete_role(RoleName=lambda_role)
iam_client.delete_policy(PolicyArn=lambda_policy)
    
lambda_client.delete_function(FunctionName=lambda_function_name_eval)

### ECR リポジトリの削除

In [ ]:
for i in container_image_list:
    ecr_client.delete_repository(
        repositoryName=i,
        force=True
    )
